In [1]:
# NOTE: This uses a modified ethereum-etl here: https://github.com/MSilb7/optimism-etl

import sys
sys.path.append("../../helper_functions")
import ethereumetl_utils as eetl
import web3py_utils as wpy
import pandas_utils as pu
sys.path.pop()

import pandas as pd
from dotenv import load_dotenv
import requests
import datetime
load_dotenv()
import os

In [2]:
configs = [
        # ['base_goerli',os.environ.get('BASE_GOERLI_NODE')]
        ['op_goerli',os.environ.get('OP_GOERLI_TEAM_NODE')]
]

max_workers = 8
start_timestamp = pu.convert_text_timestamp_to_int('2023-05-19 00:00:00')
end_timestamp = pu.convert_text_timestamp_to_int('2023-05-20 00:00:00')

In [3]:
# Get Transactions
for i in configs:

    folder = i[0]
    uri = i[1]
#     block_arr = wpy.get_blockrange_by_timedelta(i[1],trailing_hours,'hours')
    st_bl = wpy.getBlockByTimestamp(i[1],start_timestamp)# block_arr[0]
    en_bl = wpy.getBlockByTimestamp(i[1],end_timestamp) # block_arr[1]
    print(st_bl)
    print(en_bl)
    # eetl.get_ethereum_etl(st_bl,en_bl,folder,uri, max_workers)

9520367
9548129


In [4]:
# Get Receipts
for i in configs:
    folder = i[0]
    uri = i[1]
    eetl.get_eth_etl_receipts(folder, uri)

ethereumetl extract_csv_column --input downloads/op_goerli/transactions.csv --column hash --output downloads/op_goerli/transaction_hashes.txt
ethereumetl export_receipts_and_logs --transaction-hashes downloads/op_goerli/transaction_hashes.txt --provider-uri https://goerli-replica-0.optimism.io/ --receipts-output downloads/op_goerli/receipts.csv


2023-05-19 15:25:28,640 - ProgressLogger [INFO] - Started work.
2023-05-19 15:25:33,073 - ProgressLogger [INFO] - 5000 items processed.
2023-05-19 15:25:36,835 - ProgressLogger [INFO] - 10000 items processed.
2023-05-19 15:25:40,223 - ProgressLogger [INFO] - 15000 items processed.
2023-05-19 15:25:43,544 - ProgressLogger [INFO] - 20000 items processed.
2023-05-19 15:25:46,754 - ProgressLogger [INFO] - 25000 items processed.
2023-05-19 15:25:49,976 - ProgressLogger [INFO] - 30000 items processed.
2023-05-19 15:25:53,487 - ProgressLogger [INFO] - 35000 items processed.
2023-05-19 15:25:57,075 - ProgressLogger [INFO] - 40000 items processed.
2023-05-19 15:26:00,235 - ProgressLogger [INFO] - 45000 items processed.
2023-05-19 15:26:00,562 - ProgressLogger [INFO] - Finished work. Total items processed: 45033. Took 0:00:31.922500.
2023-05-19 15:26:00,562 - CompositeItemExporter [INFO] - receipt items exported: 45033
2023-05-19 15:26:00,562 - CompositeItemExporter [INFO] - log items exported: 

### Process Results

In [5]:
opg_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/transactions.csv')
rec_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/receipts.csv')
rec_df = rec_df.rename(columns={'transaction_hash':'hash'})
rec_df = rec_df[['hash','l1_gas_used','l1_fee','l1_gas_price','l1_fee_scalar']]

df = opg_df.merge(rec_df,on='hash',how='left')
#REMOVE DSEPOSITS
df = df[df['transaction_type'] != 126]
df['all_calldata_gas'] = df['l1_gas_used']
df['effective_l1_gas_used'] = df['l1_fee'] / df['l1_gas_price']
display(df)

,hash,nonce,block_hash,block_number,transaction_index,from_address,to_address,value,gas,gas_price,...,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,l1_gas_used,l1_fee,l1_gas_price,l1_fee_scalar,all_calldata_gas,effective_l1_gas_used
2,0x5610b1f4a92f09d4388d8aaa7731efa00665a8eb761a...,279181,0x17f72cbb7938528d920d551e159c467e9b54c07e4fcd...,9520967,1,0xb3311bf816cc744a86645e52a0b9d41521d0ab5c,0x43f6357305fa2ebf5aa8664a8632e49d5294390c,0,500000,2000050,...,1684470002,2000080.0,2000000.0,2,6452.0,1.021330e+13,1.194259e+09,1.0,6452.0,8552.0
3,0x48ad006ac62fbbc7851555379a0d5da76a4546128143...,260,0x17f72cbb7938528d920d551e159c467e9b54c07e4fcd...,9520967,2,0xec91d8144524674f5182ab8cbcfce0013623f3ac,0x76de22f3d8f570e3419176db9cecbb2a3f563fa0,0,12500000,2000000,...,1684470002,2000000.0,2000000.0,2,1856.0,4.724488e+12,1.194259e+09,1.0,1856.0,3956.0
5,0x63fa332edf782ed7fe53c5a22328f18ba6c00513495f...,177971,0x3c797667a259ff4bc33a4325de34f236b07dc2a67d2d...,9520968,1,0x87aa776a7686dfc124c1e7db4fd4fccfa368c18c,0xc02fad68851cf730e533a91dd2e788bef0ef57f6,0,7920027,910053666,...,1684470004,NaN,NaN,0,1752.0,4.777675e+12,1.240310e+09,1.0,1752.0,3852.0
6,0x2c1dc6c626f09a1227dff558029a4d5ef32d0e9b463d...,293176,0x3c797667a259ff4bc33a4325de34f236b07dc2a67d2d...,9520968,2,0xfa28de39d5c1aa56abb7bd07b7770c5acff0c34f,0x40a8b760a86aa0ea51bb6c2831082ab70145b9fc,0,500000,2000050,...,1684470004,2000080.0,2000000.0,2,6488.0,1.065178e+13,1.240310e+09,1.0,6488.0,8588.0
7,0x43a75b2f7a4667c4171732fde61ac57d74a7a5b4de81...,27299,0x3c797667a259ff4bc33a4325de34f236b07dc2a67d2d...,9520968,3,0xa38c9e8de9e56fae0462bdb73b20f7392681a9da,0xee85c7b5dd306f236bf13195ce0e6294764791c5,0,6000000,2000050,...,1684470004,2065958.0,2000000.0,2,8752.0,1.345985e+13,1.240310e+09,1.0,8752.0,10852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,0xe85a0904c182dfc0475944b397e194c88f863dd2b87d...,178879,0x6171fe48c63b3b7ebfb682f97300c67ad9d0a5f65c57...,9547849,1,0x87aa776a7686dfc124c1e7db4fd4fccfa368c18c,0xc02fad68851cf730e533a91dd2e788bef0ef57f6,0,7920027,910053666,...,1684523766,NaN,NaN,0,1764.0,4.731670e+14,1.224552e+11,1.0,1764.0,3864.0
45023,0x30c98820b797de6f605a56ea0ee62c091ed205b8c005...,50023,0x51543775ff0adf7cb670b108811fa5346ce6b8284f6a...,9547852,1,0x645504aca99cb631aae9d70f754c6cc0719b55bd,0x3aa03edd2dafb3c11d6f0760d1fb1a0958d0dbd0,25000000000000000,21000,100000000000,...,1684523772,NaN,NaN,0,1760.0,4.726772e+14,1.224552e+11,1.0,1760.0,3860.0
45025,0xdc969f28740c2f1b9331bdb2e7c5645bea558a3c5d18...,29268,0x79734989ed343af54d541510d88544e7052271a12e4d...,9547853,1,0xeb6304c9904dc04ef66d367b2ebc41525d1f231b,0x97597016f7dac89e55005105fc755c0513973fa8,0,383950,1800000060,...,1684523774,NaN,NaN,0,6592.0,1.064381e+15,1.224552e+11,1.0,6592.0,8692.0
45028,0xef0565d6a1b966b092cebe21b740d048640bbe53c764...,8686,0xfa2d952def503d2140bf655f1f917c53f70dd67bc99d...,9547855,1,0x86d9d510b8a8e6f6bcaa4c231a5b490956f16fea,0xa38f1c6f9f50dbd8d11add89c1a218f037498bc1,0,271529,50,...,1684523778,63.0,0.0,2,7592.0,1.141595e+15,1.177873e+11,1.0,7592.0,9692.0
